# Agenda

1. Functions
    - Inner functions
    - Dispatch tables
    - Bytecodes and compilation
2. Mypy + annotations
    - Type annotations / type hints
    - Mypy 
    - Modern techniques
3. Objects
    - What happens when we create an object?
    - Attributes (ICPO)
    - Methods 
    - Magic methods
    - Inheritance

# Inner functions

1. Functions are objects (can be arguments to functions, can be return values from functions)
2. When we use `def`, we (a) create a function object and (b) assign to a variable
3. When we assign to a variable in a function, the variable is local

In [1]:
def outer():
    def inner():
        print('I am in inner!')
    return inner

x = outer()    

In [2]:
type(x)

function

In [3]:
x.__name__

'inner'

In [4]:
x()

I am in inner!


In [5]:
def hello():
    return 'Hello!'

In [6]:
hello.__name__

'hello'

In [7]:
y = hello

In [8]:
y.__name__

'hello'

In [9]:
del(hello)

In [10]:
y()

'Hello!'

In [12]:
def outer(a):
    def inner(b):
        print(f'I am in inner {a=}, {b=}!')
    return inner

x = outer(10)

In [13]:
type(x)

function

In [14]:
x.__code__.co_varnames

('b',)

In [15]:
x.__code__.co_argcount

1

In [16]:
x(20)

I am in inner a=10, b=20!


In [17]:
y = outer(15)

In [18]:
y(20)

I am in inner a=15, b=20!


In [19]:
# closure

# Exercise: Password creator creator

1. Define a function, `create_password_creator`, that takes a string argument -- the characters you want in a potential password.
2. The function returns a new function, which takes an int argument.
3. When we call the returned function, we get a new password of the length we said, with random characters from the string we passed.

Example:

    create_number_pw = create_password_creator('12345')
    create_symbol_pw = creator_password_creator('!@#$%')

    new_number_pw = create_number_pw(9)
    new_symbol_pw = create_symbol_pw(11)

You can use `random.choice`    

In [ ]:
def create_password_creator(s):
    def create_password()